In [1]:
from sagas.ofbiz.services import OfService as s, oc, track
from sagas.ofbiz.entities import OfEntity as e
ok, r=track(lambda a: s().testScv(defaultValue=5.5, message="hello world"))
print(ok, r) 

✔ testScv default 21 2019-02-13 19:33:13.459 ➷ 3 ms
True {'responseMessage': 'success', 'resp': 'service done'}


In [14]:
services=oc.all_service_names()
print(len(services))

3551


In [10]:
def norm_loc(loc):
    pkg_prefix='ofbiz-framework/applications'
    idx=loc.find(pkg_prefix)
    return loc[idx+len(pkg_prefix)+1:].replace('servicedef/','').replace('.xml','').replace('/','_')

loc='file:/Users/xiaofeiwu/jcloud/vagrant/fedora/fedora-28/ofbiz/ofbiz-framework/applications/product/servicedef/services_price.xml'
norm_loc(loc)

'product_services_price'

In [13]:
from sagas.ofbiz.services import OfService as s, oc, track
from py4j.java_gateway import get_field

def search_service(name_filter):
    name_filter=name_filter.lower()
    services=oc.all_service_names()
    result=[]
    for name in services:
        if name_filter in name.lower():
            result.append(name)
    return result

srvs=search_service('productprice')[:3]
for srv in srvs:
    serv_model=oc.service_model(srv)
    def_loc=get_field(serv_model, 'definitionLocation')
    ns=get_field(serv_model, 'nameSpace')
    loc=get_field(serv_model, 'location') 
    
    print("➴", srv, norm_loc(def_loc))

➴ calculateProductPrice product_services_pricepromo
➴ createProductPrice product_services
➴ createProductPriceAction product_services_pricepromo


In [9]:
from py4j.java_gateway import get_field

services=oc.all_service_names()
out_types=set()
for name in services:
    serv_model=oc.service_model(name)
    params=serv_model.getModelParamList()
    for param in params:
        internal = get_field(param, "internal")
        if internal:
            pass
        elif get_field(param, 'mode')!='OUT':
            param_type=get_field(param, 'type')
            out_types.add(param_type)

print(out_types)            

{'org.apache.ofbiz.shipment.verify.VerifyPickSession', 'java.io.File', 'Timestamp', 'javax.servlet.http.HttpServletRequest', 'Integer', 'byte[]', 'Set', 'java.util.Collection', 'java.lang.Integer', 'java.io.Writer', 'org.apache.ofbiz.webapp.event.FileUploadProgressListener', 'Map', 'java.util.List', 'List', 'java.nio.ByteBuffer', 'java.sql.Date', 'org.apache.ofbiz.entity.GenericValue', 'java.util.ListIterator', 'java.io.OutputStream', 'org.apache.ofbiz.shipment.weightPackage.WeightPackageSession', 'GenericPK', 'String', 'Boolean', 'javax.servlet.http.HttpServletResponse', 'java.io.InputStream', 'Object', 'Long', 'java.math.BigDecimal', 'java.sql.Time', 'org.apache.ofbiz.entity.condition.EntityConditionList', 'GenericEntity', 'java.lang.Boolean', 'org.apache.ofbiz.order.shoppingcart.ShoppingCart', 'java.util.Map', 'org.apache.ofbiz.service.mail.MimeMessageWrapper', 'org.apache.ofbiz.product.config.ProductConfigWrapper', 'org.apache.ofbiz.shipment.packing.PackingSession', 'java.sql.Times

In [10]:
known_out_types={'String', 'java.sql.Date', 'java.lang.String',
                 'java.sql.Time', 'java.sql.Timestamp',
                 'Integer', 'Boolean','java.lang.Boolean',
                 'java.util.TimeZone','GenericPK',
                 'Timestamp',
                 # 'java.nio.ByteBuffer',
                 'java.math.BigDecimal', 'byte[]',
                 'java.lang.Integer', 'GenericEntity',
                 'GenericValue', 'java.util.Locale',
                 'java.net.URL','BigDecimal','Double','Long',
                 'org.apache.ofbiz.entity.GenericValue'}
print(len(out_types))
print(len(known_out_types))
for ot in out_types:
    if ot not in known_out_types:
        print(ot)

49
22
org.apache.ofbiz.shipment.verify.VerifyPickSession
java.io.File
javax.servlet.http.HttpServletRequest
Set
java.util.Collection
java.io.Writer
org.apache.ofbiz.webapp.event.FileUploadProgressListener
Map
java.util.List
List
java.nio.ByteBuffer
java.util.ListIterator
java.io.OutputStream
org.apache.ofbiz.shipment.weightPackage.WeightPackageSession
javax.servlet.http.HttpServletResponse
java.io.InputStream
Object
org.apache.ofbiz.entity.condition.EntityConditionList
org.apache.ofbiz.order.shoppingcart.ShoppingCart
java.util.Map
org.apache.ofbiz.service.mail.MimeMessageWrapper
org.apache.ofbiz.product.config.ProductConfigWrapper
org.apache.ofbiz.shipment.packing.PackingSession
junit.framework.Test
java.awt.image.BufferedImage
org.apache.ofbiz.order.shoppingcart.product.ProductPromoWorker$ActionResultInfo
junit.framework.TestResult
org.apache.ofbiz.entity.condition.EntityCondition


In [7]:
from sagas.ofbiz.services import OfService as s, oc, track
from sagas.ofbiz.entities import OfEntity as e

oc.import_package('java.sql.Date')
oc.import_package('java.sql.Time')
oc.import_package('java.net.URL')
oc.import_package('org.apache.ofbiz.base.util.UtilDateTime')
# timestamp = new Date().format 'yyyy-MM-dd HH:mm:ss'
ctx=oc.jmap(date=oc.j.Date.valueOf('2010-12-10'),
            time=oc.j.Time.valueOf('12:12:10'),
            timestamp=oc.j.UtilDateTime.nowTimestamp(),
           url=oc.j.URL("http://localhost:8080"))
print(ctx['date'], ctx['url'])
print(oc.j.ValueHelper.mapToJson(ctx))

2010-12-10 http://localhost:8080
{"date":1291910400000,"time":"12:12:10","url":"http://localhost:8080","timestamp":1549962394781}


In [14]:
def is_regular_service(name):
    serv_model=oc.service_model(name)
    params=serv_model.getModelParamList()
    is_regular=True
    for param in params:
        internal = get_field(param, "internal")
        p_name = get_field(param, "name")
        # print(p_name, internal)
        if internal:
            pass
        elif get_field(param, 'mode')!='OUT':
            param_type=get_field(param, 'type')
            if param_type=='java.nio.ByteBuffer':
                print("service {} parameter {}'s type is ByteBuffer"
                     .format(name, p_name))      
                is_regular=False
                break
            elif param_type not in known_out_types:
                is_regular=False
                break
    return is_regular

def get_all_regular_services():
    services=oc.all_service_names()
    regular_services=[]
    for name in services:    
        if is_regular_service(name):
            regular_services.append(name)
    return regular_services

is_regular_service('deleteProductPrice')

True

In [15]:
regular_services=get_all_regular_services()
print(len(regular_services))

service addAdditionalViewForProduct parameter uploadedFile's type is ByteBuffer
service addImageForProductPromo parameter uploadedFile's type is ByteBuffer
service addMultipleuploadForProduct parameter uploadedFile's type is ByteBuffer
service attachUploadToDataResource parameter uploadedFile's type is ByteBuffer
service buildSurveyFromPdf parameter inputByteBuffer's type is ByteBuffer
service buildSurveyResponseFromPdf parameter inputByteBuffer's type is ByteBuffer
service byteBufferTest parameter byteBuffer1's type is ByteBuffer
service byteBufferTest parameter byteBuffer2's type is ByteBuffer
service createAnonFile parameter binData's type is ByteBuffer
service createBlogEntry parameter uploadedFile's type is ByteBuffer
service createBulkProductPromoCode parameter uploadedFile's type is ByteBuffer
service createBulkProductPromoCodeEmail parameter uploadedFile's type is ByteBuffer
service createCatalinaSession parameter sessionInfo's type is ByteBuffer
service createContentFromDataRe

In [13]:
print(len(oc.all_service_names()))

3551


In [1]:
from sagas.ofbiz.services import OfService as s, search_service
search_service('billingAccount')

['calcBillingAccountBalance',
 'captureBillingAccountPayment',
 'captureBillingAccountPayments',
 'createBillingAccount',
 'createBillingAccountAndRole',
 'createBillingAccountRole',
 'createBillingAccountTerm',
 'createBillingAccountTermAttr',
 'deleteBillingAccountTermAttr',
 'refundBillingAccountPayment',
 'removeBillingAccountRole',
 'removeBillingAccountTerm',
 'updateBillingAccount',
 'updateBillingAccountRole',
 'updateBillingAccountTerm',
 'updateBillingAccountTermAttr']

In [3]:
s('meta', False).createBillingAccount

,description,entity name,internal,mode,name,override optional,required,type,parameter mode
0,None,BillingAccount,,OUT,billingAccountId,*,*,String,OUT
1,None,BillingAccount,,IN,accountLimit,*,,java.math.BigDecimal,IN
2,None,BillingAccount,,IN,accountCurrencyUomId,*,,String,IN
3,None,BillingAccount,,IN,contactMechId,*,,String,IN
4,None,BillingAccount,,IN,fromDate,*,,java.sql.Timestamp,IN
5,None,BillingAccount,,IN,thruDate,*,,java.sql.Timestamp,IN
6,None,BillingAccount,,IN,description,*,,String,IN
7,None,BillingAccount,,IN,externalAccountId,*,,String,IN
